In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.dates as md
import seaborn as sns
import datetime as datetime
import numpy as np
import os

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

import warnings 
warnings.filterwarnings('ignore')

my_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [21]:
#CORRER PRIMERO EL NOTEBOOK FEATURES PARA CREAR LOS CSV DE LABELS Y FEATURES

labels = []
features_por_ventana = []

for i in range(0,7):
    labels.append(pd.read_csv('labels_v'+str(i+1)+'.csv'))
    labels[i].drop('Unnamed: 0',axis='columns',inplace=True)
    features_por_ventana.append(pd.read_csv('features_v'+str(i+1)+'.csv'))
    features_por_ventana[i].drop('Unnamed: 0',axis='columns',inplace=True)

In [22]:
labels[0].head()

,device_id,st,sc
0,1826643666390887030,0.015050,259200.000
1,7037174172278258682,0.029014,259200.000
2,3392065368947589877,0.057540,259200.000
3,1228982273563226229,0.126828,259200.000
4,4123059034628125459,0.132510,17800.664


In [23]:
features_por_ventana[0].head()

,device_id,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23,device_os_version
0,41863526108385,35.0,0.0,0.0,0.0,0.0,1.0,34.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,135153013040192,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,161514654074162,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,181891380775191,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,186034136943920,7.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,...,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Label

In [24]:
labels_b = []
tiempo_maximo = 259200

for i in range(0,7):
    labels[i] = (labels[i].loc[labels[i]['device_id'].isin(features_por_ventana[i]['device_id'])])[['device_id','st']]
    labels[i]['binary'] = np.where(labels[i]['st']>=tiempo_maximo,False,True)
    labels[i].sort_values(by='device_id',inplace=True)
    labels_b.append(labels[i][['binary','st']].to_records(index=False))

In [25]:
labels_b[0]

rec.array([( True, 157228.465866), ( True, 187854.009137),
           ( True,  10366.357746), ..., ( True,  50088.250389),
           ( True,  81157.046477), ( True,   1081.247579)],
          dtype=[('binary', '?'), ('st', '<f8')])

# Se agrega chequeo para que los df_feautures cumplan condiciones en los algoritmos a ser usados

In [26]:
for i in range(0,7):
    if (features_por_ventana[i].shape[0] != labels[i].shape[0]):
        raise AssertionError("Los df_features de cada ventana tienen que tener misma \
                cantidad de dispositivos(mismos device_id)")

# Algoritmo

In [27]:
from sksurv.column import categorical_to_numeric
for i in range(0,7):
    features_por_ventana[i].replace([np.inf, -np.inf],0,inplace=True)
    features_por_ventana[i]['tipo_evento'] = features_por_ventana[i]['tipo_evento'].astype('category')
    features_por_ventana[i]['wifi'] = features_por_ventana[i]['wifi'].astype('category')
    features_por_ventana[i]['kind'] = features_por_ventana[i]['kind'].astype('category')
    features_por_ventana[i]['device_os_version'] = features_por_ventana[i]['device_os_version'].astype('category')
    features_por_ventana[i]['prediccion_ventana_0'] = 0
    features_por_ventana[i]['prediccion_ventana_1'] = 0
    features_por_ventana[i]['prediccion_ventana_2'] = 0
    features_por_ventana[i]['prediccion_ventana_3'] = 0
    features_por_ventana[i]['prediccion_ventana_4'] = 0
    features_por_ventana[i]['prediccion_ventana_5'] = 0
    features_por_ventana[i] = categorical_to_numeric(features_por_ventana[i])
    features_por_ventana[i].fillna(0,inplace=True)
    
    

# GradientBoostingRegressor

In [28]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

gradient_boost = GradientBoostingRegressor()

def root_mean_squared_error(y_test, predicted):
    return (np.sqrt(mean_squared_error(y_test,predicted)))

In [29]:
print(datetime.datetime.now())

2019-06-23 19:25:28.024754


In [30]:
score =[]
for i in range(0,4):
    
    x_train = features_por_ventana[i].sort_values(by='device_id',ascending=True).drop('device_id',axis='columns')
    y_train = labels[i].sort_values(by='device_id',ascending=True)['st']
    x_test =  features_por_ventana[i+3].sort_values(by='device_id',ascending=True).drop('device_id',axis='columns')
    y_test = labels[i+3].sort_values(by='device_id',ascending=True)['st']
    
    gradient_boost.fit(x_train,y_train)
    predicted_gb = gradient_boost.predict(x_test)
    features_por_ventana[i+3]['prediccion_ventana_'+str(i)] = pd.Series(predicted_gb)
    score.append(root_mean_squared_error(y_test,predicted_gb))
    
    

In [31]:
print(datetime.datetime.now())

2019-06-23 19:34:22.033634


In [32]:
score

[153927.91953471047, 154985.57187696343, 154200.09785326626, 131898.682636683]

In [12]:
from sksurv.preprocessing import OneHotEncoder

features_v1 = OneHotEncoder().fit_transform(features_por_ventana[0])
features_v2 = OneHotEncoder().fit_transform(features_por_ventana[1])
features_v1.head()

,device_id,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23,device_os_version,prediccion_ventana_0,prediccion_ventana_1,prediccion_ventana_2,prediccion_ventana_3,prediccion_ventana_4,prediccion_ventana_5
0,41863526108385,35.0,0.0,0.0,0.0,0.0,1.0,34.0,0.0,0.0,...,24.0,0.0,0.0,0,0,0,0,0,0,0
1,135153013040192,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,161514654074162,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,181891380775191,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,186034136943920,7.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


# GradientBoostingSurvivalAnalysis

In [ ]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

score = []
for i in range(0,4):
    model = GradientBoostingSurvivalAnalysis()
    model.fit(features_por_ventana[i],labels_b[i])
    #prediccion = model.predict(features_por_ventana[i+1])
    #features_por_ventana[i+1]['prediccion_ventana_'+str(i)] = pd.Series(prediccion)
    score.append( model.score(features_por_ventana[i+1], labels_b[i+1]) )

In [ ]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

model = GradientBoostingSurvivalAnalysis()

In [ ]:
model.fit(features_v1.head(5000),labels_b[0][:5000])

In [ ]:
model.score(features_v2.head(5000),labels_b[1][:5000])

In [ ]:
from sksurv.preprocessing import OneHotEncoder
for i in range(0,5):
    features_por_ventana[i] = OneHotEncoder().fit_transform(features_por_ventana[i])

In [ ]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

score = []
for i in range(0,1):
    model = GradientBoostingSurvivalAnalysis()
    model.fit(features_por_ventana[i].head(5000),labels_b[i][:5000])
    prediccion = model.predict(features_por_ventana[i+1].head(5000))
    features_por_ventana[i+1]['prediccion_ventana_'+str(i)] = prediccion
    score.append( model.score(features_por_ventana[i+1].head(5000), labels_b[i+1][:5000]) )
    
    
    

In [ ]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

estimator = CoxPHSurvivalAnalysis(alpha=0.0001)
estimator.fit(features_v1.head(10000),labels_b[0][:10000])

In [ ]:
pd.Series(estimator.coef_, index=features_v1.columns)

#### El resultado es un vector de coeficientes, uno para cada variable, donde cada valor corresponde a la relación del registro y evento

In [ ]:
estimator.score(features_v2.head(10000), labels_b[1][:10000])

# Datasets split

In [13]:
from sklearn.metrics import mean_squared_error

def root_mean_squared_error(y_test, predicted):
    return (np.sqrt(mean_squared_error(y_test,predicted)))

In [14]:
df = labels[0].sort_values(ascending=True,by='device_id').head(10000)
df.head()

,device_id,st,binary
257007,41863526108385,157228.465866,True
285766,135153013040192,187854.009137,True
70944,161514654074162,10366.357746,True
316713,181891380775191,256765.420614,True
155213,186034136943920,60166.331894,True


In [15]:
df_feat = features_v1.sort_values(ascending=True,by='device_id').head(10000)
df_feat.head()

,device_id,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23,device_os_version,prediccion_ventana_0,prediccion_ventana_1,prediccion_ventana_2,prediccion_ventana_3,prediccion_ventana_4,prediccion_ventana_5
0,41863526108385,35.0,0.0,0.0,0.0,0.0,1.0,34.0,0.0,0.0,...,24.0,0.0,0.0,0,0,0,0,0,0,0
1,135153013040192,8.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,161514654074162,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,181891380775191,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,186034136943920,7.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [16]:
from sklearn.model_selection import train_test_split

#TRAIN_TEST_SPLIT
#Split arrays or matrices into random train and test subsets
x_train, x_test, y_train, y_test = train_test_split(df_feat.drop(columns='device_id'),
                                                    df['st'],
                                                    test_size=0.2,
                                                    random_state=43)

# KNN

In [28]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

k=5 #-->Hiperparametro
neigh = KNeighborsRegressor(n_neighbors = k ,    #Cantidad de vecinos a usar --> K hiperparametro.
                            n_jobs=-1,           #Define la cantidad de procesadores a usar, -1 todos los disponibles.
                            weights='distance',  #Se define el peso como la inversa a la distancia, los mas cercanos tienen mayor influencia.
                            algorithm='brute'    #Fuerza bruta para el algorimo, podría ser KDTree o sino automatico.
                           )
#En weights se puede pasar una funcion definida por el usuario--> Segundo hiperparametro posible de KNN.
neigh.fit(x_train,y_train) #Fit the model using X as training data and Y as target values
(root_mean_squared_error(y_test, neigh.predict(x_test)), neigh.score(x_train,y_train),neigh.score(x_test,y_test))

(73539.3725672565, 0.5393407379662112, 0.4116223190865531)

# Gradient Boosting

In [27]:
from sklearn.ensemble import GradientBoostingRegressor
gradient_boost = GradientBoostingRegressor()
gradient_boost.fit(x_train,y_train)
(root_mean_squared_error(y_test, gradient_boost.predict(x_test)), gradient_boost.score(x_train,y_train),gradient_boost.score(x_test,y_test))

(24404.462550766973, 0.9372422830604865, 0.9352030080196484)

### Gradient Boosting - GridSearch

In [63]:
from sklearn.model_selection import GridSearchCV
p_test1 = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[100,250,500,750,1000,1250,1500,1750]}

tuning = GridSearchCV(estimator =GradientBoostingRegressor(), param_grid = p_test1)

tuning.fit(df_feat.drop(columns='device_id'),df['st'])
tuning.best_params_, tuning.best_score_

({'learning_rate': 0.005, 'n_estimators': 1250}, 0.9335394614324747)

In [65]:
gradient_boost_2 = GradientBoostingRegressor(n_estimators= 1250, learning_rate=0.005)
gradient_boost_2.fit(x_train,y_train)
predicted_gb_2 = gradient_boost_2.predict(x_test)
gradient_boost_2.score(x_train, y_train)
root_mean_squared_error(y_test,predicted_gb_2)

24429.581248523762

In [67]:
gradient_boost_2.score(x_test, y_test)
#Baja

0.9350695526174552

In [71]:
p_test2 = {'max_depth':[2,3,4,5,6,7]}
tuning = GridSearchCV(estimator =GradientBoostingRegressor(), param_grid = p_test2)

tuning.fit(df_feat.drop(columns='device_id'),df['st'])
tuning.best_params_, tuning.best_score_

({'max_depth': 3}, 0.9338475449027231)

In [72]:
gradient_boost_3 = GradientBoostingRegressor(max_depth= 3)
gradient_boost_3.fit(x_train,y_train)
gradient_boost_3.score(x_test, y_test)
#baja

0.9351380720519836

In [73]:
p_test3 = {'min_samples_split':[2,4,6,8,10,20,40,60,100], 'min_samples_leaf':[1,3,5,7,9] }

tuning = GridSearchCV(estimator =GradientBoostingRegressor(), param_grid = p_test3)

tuning.fit(df_feat.drop(columns='device_id'),df['st'])
tuning.best_params_, tuning.best_score_

({'min_samples_leaf': 5, 'min_samples_split': 60}, 0.9339450691357554)

In [74]:
gradient_boost_4 = GradientBoostingRegressor(min_samples_leaf= 5, min_samples_split= 60)
gradient_boost_4.fit(x_train,y_train)
gradient_boost_4.score(x_test, y_test)
#Baja

0.9350079834773051

# Random Forest

In [26]:
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor()
random_forest.fit(x_train,y_train)
(root_mean_squared_error(y_test, random_forest.predict(x_test)), random_forest.score(x_train,y_train),random_forest.score(x_test,y_test))

(26151.213795828015, 0.9637028642597366, 0.9255953544244009)

# Linear Regression

In [25]:
from sklearn.linear_model import LinearRegression
reg_lineal = LinearRegression()
reg_lineal.fit(x_train,y_train)
(root_mean_squared_error(y_test, reg_lineal.predict(x_test)), reg_lineal.score(x_train,y_train),reg_lineal.score(x_test,y_test))

(92393.60424428641, 0.06929584017408585, 0.07124713162299445)

# Ada Boost

In [24]:
from sklearn.ensemble import AdaBoostRegressor
ada_boost = AdaBoostRegressor()
ada_boost.fit(x_train,y_train)
(root_mean_squared_error(y_test, ada_boost.predict(x_test)), ada_boost.score(x_train,y_train),ada_boost.score(x_test,y_test))

(24884.160947039603, 0.9321162772062425, 0.932630650636244)

# Decision Tree

In [51]:
from sklearn.tree import DecisionTreeRegressor
arbol_dec = DecisionTreeRegressor()
arbol_dec.fit(x_train,y_train)
(root_mean_squared_error(y_test, arbol_dec.predict(x_test)), arbol_dec.score(x_train,y_train),arbol_dec.score(x_test,y_test))

### Feature importances

In [57]:
feature_importances = pd.DataFrame(arbol_dec.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances.head(10)

,importance
cantidad_subastas_Thursday,0.778212
cantidad_subastas_Friday,0.173654
cantidad_subastas_Saturday,0.023967
cantidad_de_subastas_en_ventana,0.008015
cantidad_de_eventos_en_ventana,0.001236
aplicacion_mas_usada,0.001119
tipo_evento_mas_frec,0.001060
Porc_I-E,0.000908
cantidad_instalaciones_Thursday,0.000735
cantidad_eventos_Friday,0.000727


# ExtraTreeRegressor

In [23]:
from sklearn.tree import ExtraTreeRegressor
extra_tree = ExtraTreeRegressor()
extra_tree.fit(x_train,y_train)
extra_tree.score(x_test,y_test)

0.8910753425261935

# XGBoost

In [22]:
from xgboost import XGBRegressor
xgboost = XGBRegressor()
xgboost.fit(x_train,y_train)
predicted_xg = xgboost.predict(x_test)
(root_mean_squared_error(y_test,predicted_xg), xgboost.score(x_train,y_train), xgboost.score(x_test,y_test))

[18:56:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(24445.925918910216, 0.9370968849813937, 0.9349826398070178)

### GridSearch

In [93]:
parameters_xgb = {
        'num_boost_round': [100, 250, 500],
        'eta': [0.05, 0.1, 0.3],
        'max_depth': [6, 9, 12],
        'subsample': [0.9, 1.0],
        'colsample_bytree': [0.9, 1.0],
}

tuning_xgb = GridSearchCV(estimator = XGBRegressor(), param_grid = parameters_xgb)

tuning_xgb.fit(df_feat.drop(columns='device_id'),df['st'])
tuning_xgb.best_params_, tuning.best_score_

[14:44:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:44:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:45:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:45:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

({'colsample_bytree': 0.9,
  'eta': 0.05,
  'max_depth': 6,
  'num_boost_round': 100,
  'subsample': 1.0},
 0.9339450691357554)

# Majority Voting

In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn import model_selection
#Jugando con cuales se usan se obtienen distintos resultados
estimators = []
estimators.append(('RandomForest',RandomForestRegressor()))
#estimators.append(('DecisionTree', DecisionTreeRegressor()))
estimators.append(('AdaBoost', AdaBoostRegressor()))
#estimators.append(('LinearRegression',LinearRegression()))
estimators.append(('XGBoost',XGBRegressor()))
estimators.append(('GradientBoosting',GradientBoostingRegressor()))
voting = VotingRegressor(estimators)
voting.fit(x_train,y_train)
(root_mean_squared_error(y_test, voting.predict(x_test)), voting.score(x_train,y_train),voting.score(x_test,y_test))

# Bagging 

In [20]:
from sklearn.ensemble import BaggingRegressor
bagging = BaggingRegressor()
bagging.fit(x_train,y_train)
(root_mean_squared_error(y_test, bagging.predict(x_test)), bagging.score(x_train,y_train),bagging.score(x_test,y_test))

(26283.321203105806, 0.9637006782608304, 0.9248417196319347)

# HistGradientBoostingRegressor

In [29]:
#Este sirve para grandes cantidades de parametros
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
bagging = HistGradientBoostingRegressor()
bagging.fit(x_train,y_train)
(root_mean_squared_error(y_test, bagging.predict(x_test)), bagging.score(x_train,y_train),bagging.score(x_test,y_test))

KeyboardInterrupt: 